In [1]:
# Блок 1: Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler


In [2]:
# Блок 2: Загрузка данных
data = pd.read_excel('ув 90-10.xlsx')
df = pd.DataFrame(data)

print(df)


    Ув   х1    х2    х3         х4    х5       х6   х7    х8    х9        х10
0   29  9.4  10.5  4.15  91.631206   8.0  9.90000  228  31.0  0.80  51.334268
1   28  9.4  11.0  4.20  88.652482   2.0  9.90000  565  31.0  1.20  47.393132
2   26  8.9  11.9  4.40  91.835206  16.0  9.40000  448  27.0  0.81  47.800858
3   25  8.7  12.0  4.40  90.229885   0.3  9.20000  300  28.0  0.85  49.700368
4   27  9.1  10.7  4.00  92.307692   2.0  9.60000  293  27.0  0.82  49.644961
..  ..  ...   ...   ...        ...   ...      ...  ...   ...   ...        ...
85  20  9.2  10.0  3.90  91.050725   0.1  9.82780   90  24.0  0.53  54.259687
86  27  9.6  10.5  3.90  92.152778   0.1  9.90015  470  34.4  0.64  48.644606
87  24  9.5  10.5  4.00  92.912281   0.1  9.60850  290  28.4  0.77  49.909203
88  22  9.6  11.0  3.90  90.694444   0.1  9.73890  480  29.6  0.91  47.900505
89  22  9.7  10.0  3.80  92.100000   0.1  9.74770  280  28.0  1.13  50.000000

[90 rows x 11 columns]


In [3]:
# Блок 3: Определение входных и выходных данных
headers = df.columns.tolist()
print(headers)

X = df[['х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']]
y = df['Ув']

['Ув', 'х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']


In [4]:
# Блок 4: Разделение данных на обучающую, тестовую и валидационную выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [5]:
# Нормализация данных: приведение всех признаков к одному масштабу
scaler = StandardScaler()  # Создание объекта для нормализации
X_train_scaled = scaler.fit_transform(X_train)  # Обучение и преобразование обучающего набора данных
X_test_scaled = scaler.transform(X_test)  # Преобразование тестового набора данных

In [6]:
# Блок 5: Сетка гиперпараметров и кросс-валидация
# Задание сетки гиперпараметров для модели MLPRegressor
from sklearn.model_selection import ShuffleSplit

param_grid = {
    'solver': ['sgd'],
    'activation': ['logistic'],
    'alpha': [0.95, 0.97, 1.0],
    'hidden_layer_sizes': [(8,), (9,), (10,)],
    'max_iter': [750],
    'epsilon': [1e-5, 1e-6, 1e-7],
    'learning_rate': ['constant', 'invscaling', 'adaptive'], # Разные стратегии
    'learning_rate_init': [0.05, 0.1, 0.5], # Начальная скорость
    'momentum': [0.3, 0.5, 0.7], # Инерция
    'nesterovs_momentum': [True], # Обычно улучшает сходимость
    'tol': [0.0001, 0.001, 0.01], # Допуск для остановки
    'n_iter_no_change': [10, 15], # Для ранней остановки с adaptive
    'early_stopping': [True]
}

# Настройка кросс-валидации с 5 фолдами
cv = ShuffleSplit(
    n_splits=5,         # количество фолдов
    test_size=0.25,       # перемешивание данных
    random_state=42     # для воспроизводимости результатов
)

# GridSearchCV для подбора гиперпараметров модели
grid_search = GridSearchCV(
    estimator=MLPRegressor(random_state=42),  # Использование модели MLPRegressor
    param_grid=param_grid,  # Сетка гиперпараметров для перебора
    scoring='neg_mean_absolute_percentage_error',
    cv=cv,  # Кросс-валидация
    verbose=2,  # Вывод подробной информации о процессе подбора
    n_jobs=-1  # Использование всех доступных ядер для ускорения работы
)


In [7]:
# Блок 6: Обучение GridSearchCV
print("Начинается подбор гиперпараметров...")
grid_search.fit(X_train_scaled, y_train)  # Обучение модели с подбором гиперпараметров

Начинается подбор гиперпараметров...
Fitting 5 folds for each of 4374 candidates, totalling 21870 fits


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=42, test_size=0.25, train_size=None),
             estimator=MLPRegressor(random_state=42), n_jobs=-1,
             param_grid={'activation': ['logistic'], 'alpha': [0.95, 0.97, 1.0],
                         'early_stopping': [True],
                         'epsilon': [1e-05, 1e-06, 1e-07],
                         'hidden_layer_sizes': [(8,), (9,), (10,)],
                         'learning_rate': ['constant', 'invscaling',
                                           'adaptive'],
                         'learning_rate_init': [0.05, 0.1, 0.5],
                         'max_iter': [750], 'momentum': [0.3, 0.5, 0.7],
                         'n_iter_no_change': [10, 15],
                         'nesterovs_momentum': [True], 'solver': ['sgd'],
                         'tol': [0.0001, 0.001, 0.01]},
             scoring='neg_mean_absolute_percentage_error', verbose=2)

In [8]:
# Блок 7: Лучшая модель
best_params = grid_search.best_params_  # Получение лучших гиперпараметров

# Получение модели с лучшими параметрами
best_model = grid_search.best_estimator_

# Прогнозирование на тестовом наборе данных с лучшей моделью
y_pred_best = best_model.predict(X_test_scaled)

In [9]:
# Блок 8: Вычисление метрик
# Вычисление MAPE (Mean Absolute Percentage Error) для оценки точности
mape_best = mean_absolute_percentage_error(y_test, y_pred_best) * 100
# Вычисление RMSE (Root Mean Squared Error) для оценки отклонений предсказаний от реальных значений
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

print("Лучшие параметры:", grid_search.best_params_)
print("Обучающая выборка:")
print("Лучший результат:", -grid_search.best_score_ * 100, "%")
# Вывод лучших метрик на кросс-валидации
print("Тестовая выборка:")
print(f"MAPE: {mape_best:.2f}%")
print(f"RMSE: {rmse_best:.2f}")

Лучшие параметры: {'activation': 'logistic', 'alpha': 1.0, 'early_stopping': True, 'epsilon': 1e-05, 'hidden_layer_sizes': (9,), 'learning_rate': 'invscaling', 'learning_rate_init': 0.05, 'max_iter': 750, 'momentum': 0.5, 'n_iter_no_change': 15, 'nesterovs_momentum': True, 'solver': 'sgd', 'tol': 0.0001}
Обучающая выборка:
Лучший результат: 8.008334169782309 %
Тестовая выборка:
MAPE: 7.98%
RMSE: 2.47


In [10]:
# Блок 9: Создание DataFrame для тестовых значений
# Сравнение истинных значений с предсказанными для тестового набора
test_comparison_model = pd.DataFrame({"yв": y_test.values, "y_pred_best": y_pred_best})
print(test_comparison_model)

    yв  y_pred_best
0   19    21.846917
1   23    23.902611
2   25    22.566873
3   23    23.258294
4   24    24.230854
5   26    24.545534
6   25    21.895490
7   22    23.786547
8   29    22.970459
9   28    24.863303
10  21    19.523881
11  26    23.962167
12  25    24.065888
13  27    24.888190
14  22    22.559180
15  21    21.393286
16  24    21.755081
17  28    23.923662


In [11]:
# Блок 10: Сохранение результатов
# Сохранение результатов и метрик в Excel файл
test_comparison_best = pd.DataFrame({"ув": y_test.values, "y_pred_py": y_pred_best})
metrics_best = pd.DataFrame({"Метрика": ["MAPE", "RMSE"], "Значение": [mape_best, rmse_best]})

# Запись в Excel
output_file = 'models_results_optimized_ув.xlsx'  # Имя выходного файла
with pd.ExcelWriter(output_file) as writer:
    test_comparison_best.to_excel(writer, sheet_name='Результаты лучшей модели', index=False)  # Результаты модели
    metrics_best.to_excel(writer, sheet_name='Метрики лучшей модели', index=False)  # Метрики модели